In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [ ]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

In [10]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [8]
hungarian_thresholds = [15,25,35,45]
minimum_detections = [25,35]

In [ ]:
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

In [ ]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-1-1", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-1-1",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

In [13]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [ ]:
#experiment.stage_experiment()

In [3]:
experiment.launch_experiment()

In [2]:
import pandas as pd

In [7]:
data = pd.DataFrame({'mota':[.641,.645,.645,.649,.625,.656,.66,.663,.855,.858,.859,.859,.862,.857,.857,.855,.791,.793
                             ,.792,.792,.791,.792,.791,.791,.768,.767,.769,.766,.766,.770,.768,.767],
              'lt':[8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],
              'ht':[45,45,35,35,25,25,15,15,45,25,25,15,15,35,35,45,45,25,25,15,45,35,35,15,45,35,25,15,45,35,25,15],
              'det':[25,35,25,35,25,35,25,35,25,35,25,35,25,35,25,35,25,35,25,25,35,35,25,35,35,25,35,35,25,35,25,25],
             'game':['ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','ER0429','PP0214','PP0214',
                     'PP0214','PP0214','PP0214','PP0214','PP0214','PP0214','ChvUtd','ChvUtd','ChvUtd','ChvUtd',
                    'ChvUtd','ChvUtd','ChvUtd','ChvUtd','PS0304','PS0304','PS0304','PS0304','PS0304','PS0304'
                    ,'PS0304','PS0304']})

In [8]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections
ht lt det                                      
15 8  25   0.77025     8         15          25
      35   0.76975     8         15          35
25 8  25   0.76100     8         25          25
      35   0.76900     8         25          35
35 8  25   0.76500     8         35          25
      35   0.76700     8         35          35
45 8  25   0.76325     8         45          25
      35   0.76475     8         45          35

In [12]:
scat_lt_ht_det_1 = Scatter(data_lt_ht_det, x='hungarian', y='mota', color='detections', marker='lost')
show(scat_lt_ht_det_1)

In [54]:
data[data['ht']==15][data[data['ht']==15]['det']==25]

,det,game,ht,lt,mota
6,25,ER0429,15,8,0.660
12,25,PP0214,15,8,0.862
19,25,ChvUtd,15,8,0.792
31,25,PS0304,15,8,0.767
